In [27]:
import numpy as np
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

In [28]:
def find_loops(edges):
    """
    Find all closed loops starting and ending at node 0 in a directed graph.

    Args:
        edges: List of tuples representing directed edges (from_node, to_node)

    Returns:
        List of lists, where each sublist contains tuples forming a closed loop from 0 to 0
    """
    # Create adjacency list for faster lookup
    adj = {}
    for start, end in edges:
        if start not in adj:
            adj[start] = []
        adj[start].append(end)

    def find_paths(current, path, visited):
        """
        Recursive helper function to find all paths from current node back to 0.
        """
        if current == 'Depot' and len(path) > 0:  # Found a loop back to 0
            return [path[:]]

        if current not in adj:  # Dead end
            return []

        paths = []
        for next_node in adj[current]:
            if (current, next_node) not in visited:  # Avoid using same edge twice
                path.append((current, next_node))
                visited.add((current, next_node))
                paths.extend(find_paths(next_node, path, visited))
                path.pop()
                visited.remove((current, next_node))
        return paths

    # Start DFS from node 0
    all_loops = find_paths('Depot', [], set())

    # Filter out non-minimal loops (loops that contain other loops)
    minimal_loops = []
    for loop in all_loops:
        # Check if this loop contains any other loop
        is_minimal = True
        visited_nodes = {node for edge in loop for node in edge}
        for other_loop in all_loops:
            if loop != other_loop:
                other_nodes = {node for edge in other_loop for node in edge}
                if other_nodes.issubset(visited_nodes) and len(other_loop) < len(loop):
                    is_minimal = False
                    break
        if is_minimal:
            minimal_loops.append(loop)

    return minimal_loops

## Cost matrix

In [29]:
cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

## Shop demands

In [30]:
shop_demands = pd.read_csv('2015_shop_locations.csv').set_index('id')[['demand(kg)', 'stage']].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'].isin([1, 2, 3]), 'demand(kg)']
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

shop_demands.shape

(100,)

## Vehicle routing problem

In [31]:
# Vehicle capacity
van_capacity = 300

In [32]:
# Creates a list of tuples containing all the possible routes for transport
A = [(i,j) for i in nodes for j in nodes if i!=j]

In [33]:
A

[('Depot', 'ChIJwTC7A55ZwokRPNX9g7ngbaI'),
 ('Depot', 'ChIJU1XImaNZwokRutunetC8XeE'),
 ('Depot', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
 ('Depot', 'ChIJ5cPkuBtgwokRn55JgpGqjFA'),
 ('Depot', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
 ('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
 ('Depot', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
 ('Depot', 'ChIJ_eQYpFdmwokR756MAH2tZPw'),
 ('Depot', 'ChIJYzaRC2REwokRaH2rHpflSYk'),
 ('Depot', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
 ('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'),
 ('Depot', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'),
 ('Depot', 'ChIJ7-uIlEFEwokRlbHGtjrdFtM'),
 ('Depot', 'ChIJU3IVO-tZwokRx6iWrhKQTng'),
 ('Depot', 'ChIJRR7W62BFwokRlrVks40mAdU'),
 ('Depot', 'ChIJnZ4lShdkwokRQUAn5mjU1pA'),
 ('Depot', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
 ('Depot', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'),
 ('Depot', 'ChIJt5DcjLRFwokRyxil5Pjp3Fo'),
 ('Depot', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
 ('Depot', 'ChIJzRG6yrrzwokRQ5kjG2RJCwM'),
 ('Depot', 'ChIJ9X0DD_FhwokRIqyyecrKeMQ'),
 ('Depot', 'ChIJfxyTIQ32wokRxjovn0UE72M'),
 ('Depot', 

In [34]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for shop in shops:
    u[shop] = LpVariable("u(%s)" %shop, shop_demands[shop], van_capacity)

In [35]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [36]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix.loc[i,j]*x[i,j] for (i,j) in A)

In [37]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for shop in shops:
    one_visit_out[shop] = lpSum([x[shop,node] for node in nodes if shop!=node]) == 1.0
    prob += one_visit_out[shop]

In [38]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for shop in shops:
    one_visit_in[shop] = lpSum([x[node,shop] for node in nodes if node!=shop]) == 1.0
    prob += one_visit_in[shop]

In [39]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot
depot_node = lpSum([x['Depot',shop] for shop in shops]) == lpSum([x[shop,'Depot'] for shop in shops])
prob += depot_node

In [40]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for shop_out in shops:
    for shop_in in shops:
        if shop_out != shop_in:
            sub_tours_cap[shop_out,shop_in] = u[shop_out] - u[shop_in] + van_capacity * x[shop_out,shop_in] <= van_capacity - shop_demands[shop_in]
            prob += sub_tours_cap[shop_out,shop_in]

In [ ]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=True, timeLimit=600))

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 10101 rows, 10200 columns and 49900 nonzeros
Model fingerprint: 0x05fd1b8f
Variable types: 100 continuous, 10100 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e+00, 3e+02]
Presolve time: 0.05s
Presolved: 10101 rows, 10200 columns, 49900 nonzeros
Variable types: 100 continuous, 10100 integer (10100 binary)
Found heuristic solution: objective 1597.7419365
Found heuristic solution: objective 1587.1870836

Root relaxation: objective 1.726494e+02, 531 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Ob

In [16]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

Status: Not Solved


In [17]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

x(Depot,ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 1.0
x(Depot,ChIJzRG6yrrzwokRQ5kjG2RJCwM) = 1.0
x(Depot,ChIJ0zAa911ZwokRQlmIU8v4Kzs) = 1.0
x(Depot,ChIJLXqvAAJdwokRSeu3qp3b1es) = 1.0
x(Depot,ChIJ_QfagF9cwokR7b8aZshUano) = 1.0
x(ChIJwTC7A55ZwokRPNX9g7ngbaI,Depot) = 1.0
x(ChIJU1XImaNZwokRutunetC8XeE,ChIJU3IVO_tZwokRx6iWrhKQTng) = 1.0
x(ChIJG7L_TLVbwokRT36uIrwz2Mo,ChIJ6ZQpRmBawokR0kz4HEWfI7U) = 1.0
x(ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ2RQH_bBhwokRTxgg6q8oot0) = 1.0
x(ChIJ24V7r_31wokR_S71l2zqrwc,ChIJ3R85DSb2wokR1XobETRMs5E) = 1.0
x(ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJ3eHXlHZZwokRuzME_28niVI) = 1.0
x(ChIJYTxm0Gf2wokR5V0iFDQ_2x0,ChIJfxyTIQ32wokRxjovn0UE72M) = 1.0
x(ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJN4cpNB1mwokRoz2EEGXJsPM) = 1.0
x(ChIJYzaRC2REwokRaH2rHpflSYk,ChIJIYrXpHdEwokRlW_Ep7hr84A) = 1.0
x(ChIJ3R85DSb2wokR1XobETRMs5E,ChIJYTxm0Gf2wokR5V0iFDQ_2x0) = 1.0
x(ChIJ97BgiDtfwokRsbkP6Vl1seY,Depot) = 1.0
x(ChIJT1ZEl4JYwokRXQMvGg9VMPE,ChIJ97BgiDtfwokRsbkP6Vl1seY) = 1.0
x(ChIJ7_uIlEFEwokRlbHGtjrdFtM,ChIJt5DcjLRFwokRyxi

In [18]:
# Print bounds
print("Lower bound = ", value(prob.solverModel.ObjBound))
print("Upper bound = ", value(prob.solverModel.ObjVal))

Lower bound =  114.15569907604565
Upper bound =  153.14526228382005


In [19]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
find_loops(active_arcs)


[[('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
  ('ChIJiUJ1DI5ZwokRWdK6SPg9BOY', 'ChIJ3eHXlHZZwokRuzME-28niVI'),
  ('ChIJ3eHXlHZZwokRuzME-28niVI', 'ChIJdTlWmaBZwokRgtKH8NPb67o'),
  ('ChIJdTlWmaBZwokRgtKH8NPb67o', 'ChIJHy0ZlqNZwokRABxBCQR_37o'),
  ('ChIJHy0ZlqNZwokRABxBCQR_37o', 'ChIJBVgKiaZYwokRmMiA0bhFd4A'),
  ('ChIJBVgKiaZYwokRmMiA0bhFd4A', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
  ('ChIJ24V7r_31wokR-S71l2zqrwc', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
  ('ChIJ3R85DSb2wokR1XobETRMs5E', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
  ('ChIJYTxm0Gf2wokR5V0iFDQ-2x0', 'ChIJfxyTIQ32wokRxjovn0UE72M'),
  ('ChIJfxyTIQ32wokRxjovn0UE72M', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
  ('ChIJG9cUYg_1wokRUNpvwM84Z4M', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
  ('ChIJ2b9se6f1wokRUN8kWpFfzN0', 'ChIJeRxpSKhfwokRD-kwzfT0mP4'),
  ('ChIJeRxpSKhfwokRD-kwzfT0mP4', 'Depot')],
 [('Depot', 'ChIJzRG6yrrzwokRQ5kjG2RJCwM'),
  ('ChIJzRG6yrrzwokRQ5kjG2RJCwM', 'ChIJj-Dtqb_zwokRYlU6uDe6ya4'),
  ('ChIJj-Dtqb_zwokRYlU6uDe6ya4', 'ChIJ96djgPjzwokRQU6pStHOAVc'),
  ('ChIJ9

In [20]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

u(ChIJwTC7A55ZwokRPNX9g7ngbaI) = 300.0
u(ChIJU1XImaNZwokRutunetC8XeE) = 248.0
u(ChIJG7L_TLVbwokRT36uIrwz2Mo) = 242.0
u(ChIJ5cPkuBtgwokRn55JgpGqjFA) = 184.0
u(ChIJ24V7r_31wokR_S71l2zqrwc) = 138.0
u(ChIJiUJ1DI5ZwokRWdK6SPg9BOY) = 15.0
u(ChIJYTxm0Gf2wokR5V0iFDQ_2x0) = 181.0
u(ChIJ_eQYpFdmwokR756MAH2tZPw) = 57.0
u(ChIJYzaRC2REwokRaH2rHpflSYk) = 75.0
u(ChIJ3R85DSb2wokR1XobETRMs5E) = 163.0
u(ChIJ97BgiDtfwokRsbkP6Vl1seY) = 292.0
u(ChIJT1ZEl4JYwokRXQMvGg9VMPE) = 273.0
u(ChIJ7_uIlEFEwokRlbHGtjrdFtM) = 127.0
u(ChIJU3IVO_tZwokRx6iWrhKQTng) = 272.0
u(ChIJRR7W62BFwokRlrVks40mAdU) = 190.0
u(ChIJnZ4lShdkwokRQUAn5mjU1pA) = 215.0
u(ChIJG9cUYg_1wokRUNpvwM84Z4M) = 231.0
u(ChIJmZQsMgFbwokRMc5I7kPdQ7w) = 297.0
u(ChIJt5DcjLRFwokRyxil5Pjp3Fo) = 148.0
u(ChIJ2b9se6f1wokRUN8kWpFfzN0) = 259.0
u(ChIJzRG6yrrzwokRQ5kjG2RJCwM) = 23.0
u(ChIJ9X0DD_FhwokRIqyyecrKeMQ) = 247.0
u(ChIJfxyTIQ32wokRxjovn0UE72M) = 204.0
u(ChIJoQK_Y1hnwokRLkhjEpVsGxo) = 136.0
u(ChIJ3eHXlHZZwokRuzME_28niVI) = 46.0
u(ChIJ0zAa911ZwokRQlmIU8v4Kzs)

In [22]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(active_arcs, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('routings/routing_optimal_2015_stage_3.csv', index=False)